# Data mining and cleaning Airbnb

In [145]:
using CSV
using Plots
using DataFrames
using Dates

There are 4 starting files:
* 2019_july_listings.csv
* 2019_october_listings.csv
* 2020_july_listings.csv
* 2020_octuber_listings.csv

Which have been altered and tidied in excel from the original files from [here](http://insideairbnb.com/get-the-data.html)

Inidividual analysis will be done to each one in case there are particular problems in any one of them. The result of this analysis is to produce 4 clean datasets that have the necessary information to work on the problem

### October 2020

In [146]:
df = CSV.read("2020_october_listings.csv");

Take a look at variables and see if they have missing values

In [147]:
feature_names = names(df);

In [148]:
for i in 1:length(feature_names)
    println(string(i), "\t", string(feature_names[i]), "\t\t\t", string(eltype(df[!, i])))
end

1	id			Int64
2	name			Union{Missing, String}
3	host_id			Int64
4	host_name			Union{Missing, String}
5	neighbourhood_group			String
6	neighbourhood			String
7	latitude			Float64
8	longitude			Float64
9	room_type			String
10	price			Int64
11	minimum_nights			Int64
12	number_of_reviews			Int64
13	last_review			Union{Missing, Date}
14	reviews_per_month			Union{Missing, Float64}
15	calculated_host_listings_count			Int64
16	availability_365			Int64


There are missing values in some categories. We will see how much they affect and how many they are.

Number of rows in full dataset

In [149]:
n_rows_full = size(df,1)

44666

In [150]:
function percentage(fraction, total)
    return fraction/total
end

percentage (generic function with 1 method)

Will see for each of the columns the percentage of missing values that there are in them. The goal of this is to see of it makes sense to substitute missing values for any dummy variable or if we can eliminate the rows.

In [151]:
for feature in feature_names
    index_missing_data = ismissing.(df[:, feature]) # For each column we obtain 1 if missing, 0 if not
    
    number_missing_data = sum(index_missing_data)
    
    # Percentage is calculated
    per = percentage(number_missing_data, n_rows_full)
    
    print("\n\n", feature, " --> ", round(per*100, digits = 3) ,"% missing values")
end



id --> 0.0% missing values

name --> 0.034% missing values

host_id --> 0.0% missing values

host_name --> 0.038% missing values

neighbourhood_group --> 0.0% missing values

neighbourhood --> 0.0% missing values

latitude --> 0.0% missing values

longitude --> 0.0% missing values

room_type --> 0.0% missing values

price --> 0.0% missing values

minimum_nights --> 0.0% missing values

number_of_reviews --> 0.0% missing values

last_review --> 23.546% missing values

reviews_per_month --> 23.546% missing values

calculated_host_listings_count --> 0.0% missing values

availability_365 --> 0.0% missing values

The results show which columns to investigate more:
* last_review
* reviews_per_month

For the rest of the columns, missing values can be eliminated

In [152]:
size(df)

(44666, 16)

`last_review` is a date column, `reviews_per_month` is a float that indicates average number of reviews per month. `Missing`values will be substituted by dummy date "01/01/1900" and value 0 respectively.

In [153]:
df[1:10, :last_review]

10-element Array{Union{Missing, Date},1}:
 2019-11-04
 2020-09-20
 2019-12-02
 2014-01-02
 2020-03-15
 2017-07-21
 2019-08-10
 2020-09-09
 2019-12-09
 2020-03-16

In [154]:
dummy_date = Date(Dates.Month(1),Dates.Year(1900))

1900-01-01

In [155]:
df[:, :last_review] = coalesce.(df[:, :last_review], dummy_date);

In [156]:
df[:, :reviews_per_month] =  coalesce.(df[:, :reviews_per_month], 0);

Because the missing values are `host_name`and `name`they will be elimnated as they representa a very small percentage of the total observations

In [157]:
index_host_name = .!ismissing.(df[:, :host_name]); # 1 to rows with non missing values

In [158]:
size(df)

(44666, 16)

In [159]:
df_clean_host_name = df[index_host_name, :];

In [160]:
size(df_clean_host_name)

(44649, 16)

Now it will be checked if there are still rows with missing `name`

In [161]:
index_name = .!ismissing.(df_clean_host_name[:, :name]); # 1 to rows with non missing values

In [162]:
sum(index_name)

44634

As this number is less than the number of rows of `df_clean_host_name` cleaning is still needed

In [163]:
df_clean = df_clean_host_name[index_name, :];

In [164]:
size(df_clean)

(44634, 16)

Final check to see all variables are ok

In [165]:
 any_missing = colwise(x -> any(ismissing.(x)), df_clean);
sum(any_missing)

0

No values are missing from any of the columns. Clean dataframe is stored in new variable to prevent overwriting

In [166]:
october_2020_dataframe = df_clean;

### July 2020

In [167]:
df = CSV.read("2020_july_listings.csv");

Take a look at variables and see if they have missing values

In [168]:
feature_names = names(df);

In [169]:
for i in 1:length(feature_names)
    println(string(i), "\t", string(feature_names[i]), "\t\t\t", string(eltype(df[!, i])))
end

1	id			Int64
2	name			Union{Missing, String}
3	host_id			Int64
4	host_name			Union{Missing, String}
5	neighbourhood_group			String
6	neighbourhood			String
7	latitude			Float64
8	longitude			Float64
9	room_type			String
10	price			Int64
11	minimum_nights			Int64
12	number_of_reviews			Int64
13	last_review			Union{Missing, Date}
14	reviews_per_month			Union{Missing, Float64}
15	calculated_host_listings_count			Int64
16	availability_365			Int64


There are missing values in some categories. We will see how much they affect and how many they are.

Number of rows in full dataset

In [170]:
n_rows_full = size(df,1)

48588

In [171]:
function percentage(fraction, total)
    return fraction/total
end

percentage (generic function with 1 method)

Will see for each of the columns the percentage of missing values that there are in them. The goal of this is to see of it makes sense to substitute missing values for any dummy variable or if we can eliminate the rows.

In [172]:
for feature in feature_names
    index_missing_data = ismissing.(df[:, feature]) # For each column we obtain 1 if missing, 0 if not
    
    number_missing_data = sum(index_missing_data)
    
    # Percentage is calculated
    per = percentage(number_missing_data, n_rows_full)
    
    print("\n\n", feature, " --> ", round(per*100, digits = 3) ,"% missing values")
end



id --> 0.0% missing values

name --> 0.033% missing values

host_id --> 0.0% missing values

host_name --> 0.025% missing values

neighbourhood_group --> 0.0% missing values

neighbourhood --> 0.0% missing values

latitude --> 0.0% missing values

longitude --> 0.0% missing values

room_type --> 0.0% missing values

price --> 0.0% missing values

minimum_nights --> 0.0% missing values

number_of_reviews --> 0.0% missing values

last_review --> 23.376% missing values

reviews_per_month --> 23.376% missing values

calculated_host_listings_count --> 0.0% missing values

availability_365 --> 0.0% missing values

Results are similar to before. We need to investigate same columns:
* last_review
* reviews_per_month

For the rest of the columns, missing values can be eliminated

In [173]:
size(df)

(48588, 16)

`last_review` is a date column, `reviews_per_month` is a float that indicates average number of reviews per month. `Missing`values will be substituted by dummy date "01/01/1900" and value 0 respectively.

In [174]:
df[1:10, :last_review]

10-element Array{Union{Missing, Date},1}:
 2008-09-22
 2019-11-04
 2020-06-21
 2016-12-23
 2019-10-13
 2019-12-02
 2014-01-02
 2020-03-15
 2017-07-21
 2019-07-29

In [175]:
dummy_date = Date(Dates.Month(1),Dates.Year(1900))

1900-01-01

In [176]:
df[:, :last_review] = coalesce.(df[:, :last_review], dummy_date);

In [177]:
df[:, :reviews_per_month] =  coalesce.(df[:, :reviews_per_month], 0);

Because the missing values are `host_name`and `name`they will be elimnated as they representa a very small percentage of the total observations

In [178]:
index_host_name = .!ismissing.(df[:, :host_name]); # 1 to rows with non missing values

In [179]:
size(df)

(48588, 16)

In [180]:
df_clean_host_name = df[index_host_name, :];

In [181]:
size(df_clean_host_name)

(48576, 16)

Now it will be checked if there are still rows with missing `name`

In [182]:
index_name = .!ismissing.(df_clean_host_name[:, :name]); # 1 to rows with non missing values

In [183]:
sum(index_name)

48560

As this number is less than the number of rows of `df_clean_host_name` cleaning is still needed

In [184]:
df_clean = df_clean_host_name[index_name, :];

In [185]:
size(df_clean)

(48560, 16)

Final check to see all variables are ok

In [186]:
 any_missing = colwise(x -> any(ismissing.(x)), df_clean);
sum(any_missing)

0

No values are missing from any of the columns. Clean dataframe is stored in new variable to prevent overwriting

In [187]:
july_2020_dataframe = df_clean;

### October 2019

In [190]:
df = CSV.read("2019_october_listings.csv");

Take a look at variables and see if they have missing values

In [191]:
feature_names = names(df);

In [192]:
for i in 1:length(feature_names)
    println(string(i), "\t", string(feature_names[i]), "\t\t\t", string(eltype(df[!, i])))
end

1	id			Int64
2	name			Union{Missing, String}
3	host_id			Int64
4	host_name			Union{Missing, String}
5	neighbourhood_group			String
6	neighbourhood			String
7	latitude			Float64
8	longitude			Float64
9	room_type			String
10	price			Int64
11	minimum_nights			Int64
12	number_of_reviews			Int64
13	last_review			Union{Missing, Date}
14	reviews_per_month			Union{Missing, Float64}
15	calculated_host_listings_count			Int64
16	availability_365			Int64


There are missing values in some categories. We will see how much they affect and how many they are.

Number of rows in full dataset

In [193]:
n_rows_full = size(df,1)

48602

In [194]:
function percentage(fraction, total)
    return fraction/total
end

percentage (generic function with 1 method)

Will see for each of the columns the percentage of missing values that there are in them. The goal of this is to see of it makes sense to substitute missing values for any dummy variable or if we can eliminate the rows.

In [195]:
for feature in feature_names
    index_missing_data = ismissing.(df[:, feature]) # For each column we obtain 1 if missing, 0 if not
    
    number_missing_data = sum(index_missing_data)
    
    # Percentage is calculated
    per = percentage(number_missing_data, n_rows_full)
    
    print("\n\n", feature, " --> ", round(per*100, digits = 3) ,"% missing values")
end



id --> 0.0% missing values

name --> 0.035% missing values

host_id --> 0.0% missing values

host_name --> 0.066% missing values

neighbourhood_group --> 0.0% missing values

neighbourhood --> 0.0% missing values

latitude --> 0.0% missing values

longitude --> 0.0% missing values

room_type --> 0.0% missing values

price --> 0.0% missing values

minimum_nights --> 0.0% missing values

number_of_reviews --> 0.0% missing values

last_review --> 19.133% missing values

reviews_per_month --> 19.133% missing values

calculated_host_listings_count --> 0.0% missing values

availability_365 --> 0.0% missing values

Results are similar to before. We need to investigate same columns:
* last_review
* reviews_per_month

For the rest of the columns, missing values can be eliminated

In [196]:
size(df)

(48602, 16)

`last_review` is a date column, `reviews_per_month` is a float that indicates average number of reviews per month. `Missing`values will be substituted by dummy date "01/01/1900" and value 0 respectively.

In [198]:
df[1:10, :last_review]

10-element Array{Union{Missing, Date},1}:
 2019-09-24
 2018-11-19
 2019-09-25
 2017-10-05
 2019-09-28
 2017-07-21
 2019-07-29
 2019-08-03
 2019-10-13
 2019-09-16

In [199]:
dummy_date = Date(Dates.Month(1),Dates.Year(1900))

1900-01-01

In [200]:
df[:, :last_review] = coalesce.(df[:, :last_review], dummy_date);

In [201]:
df[:, :reviews_per_month] =  coalesce.(df[:, :reviews_per_month], 0);

Because the missing values are `host_name`and `name`they will be elimnated as they representa a very small percentage of the total observations

In [202]:
index_host_name = .!ismissing.(df[:, :host_name]); # 1 to rows with non missing values

In [203]:
size(df)

(48602, 16)

In [204]:
df_clean_host_name = df[index_host_name, :];

In [205]:
size(df_clean_host_name)

(48570, 16)

Now it will be checked if there are still rows with missing `name`

In [206]:
index_name = .!ismissing.(df_clean_host_name[:, :name]); # 1 to rows with non missing values

In [207]:
sum(index_name)

48553

As this number is less than the number of rows of `df_clean_host_name` cleaning is still needed

In [208]:
df_clean = df_clean_host_name[index_name, :];

In [209]:
size(df_clean)

(48553, 16)

Final check to see all variables are ok

In [210]:
 any_missing = colwise(x -> any(ismissing.(x)), df_clean);
sum(any_missing)

0

No values are missing from any of the columns. Clean dataframe is stored in new variable to prevent overwriting

In [211]:
october_2019_dataframe = df_clean;

### July 2019

In [215]:
df = CSV.read("2019_july_listings.csv");

Take a look at variables and see if they have missing values

In [216]:
feature_names = names(df);

In [217]:
for i in 1:length(feature_names)
    println(string(i), "\t", string(feature_names[i]), "\t\t\t", string(eltype(df[!, i])))
end

1	id			Int64
2	name			Union{Missing, String}
3	host_id			Int64
4	host_name			Union{Missing, String}
5	neighbourhood_group			String
6	neighbourhood			String
7	latitude			Float64
8	longitude			Float64
9	room_type			String
10	price			Int64
11	minimum_nights			Int64
12	number_of_reviews			Int64
13	last_review			Union{Missing, Date}
14	reviews_per_month			Union{Missing, Float64}
15	calculated_host_listings_count			Int64
16	availability_365			Int64


There are missing values in some categories. We will see how much they affect and how many they are.

Number of rows in full dataset

In [218]:
n_rows_full = size(df,1)

48895

In [219]:
function percentage(fraction, total)
    return fraction/total
end

percentage (generic function with 1 method)

Will see for each of the columns the percentage of missing values that there are in them. The goal of this is to see of it makes sense to substitute missing values for any dummy variable or if we can eliminate the rows.

In [220]:
for feature in feature_names
    index_missing_data = ismissing.(df[:, feature]) # For each column we obtain 1 if missing, 0 if not
    
    number_missing_data = sum(index_missing_data)
    
    # Percentage is calculated
    per = percentage(number_missing_data, n_rows_full)
    
    print("\n\n", feature, " --> ", round(per*100, digits = 3) ,"% missing values")
end



id --> 0.0% missing values

name --> 0.033% missing values

host_id --> 0.0% missing values

host_name --> 0.043% missing values

neighbourhood_group --> 0.0% missing values

neighbourhood --> 0.0% missing values

latitude --> 0.0% missing values

longitude --> 0.0% missing values

room_type --> 0.0% missing values

price --> 0.0% missing values

minimum_nights --> 0.0% missing values

number_of_reviews --> 0.0% missing values

last_review --> 20.558% missing values

reviews_per_month --> 20.558% missing values

calculated_host_listings_count --> 0.0% missing values

availability_365 --> 0.0% missing values

Results are similar to before. We need to investigate same columns:
* last_review
* reviews_per_month

For the rest of the columns, missing values can be eliminated

In [221]:
size(df)

(48895, 16)

`last_review` is a date column, `reviews_per_month` is a float that indicates average number of reviews per month. `Missing`values will be substituted by dummy date "01/01/1900" and value 0 respectively.

In [222]:
df[1:10, :last_review]

10-element Array{Union{Missing, Date},1}:
 2018-10-19
 2019-05-21
 missing
 2019-07-05
 2018-11-19
 2019-06-22
 2017-10-05
 2019-06-24
 2017-07-21
 2019-06-09

In [223]:
dummy_date = Date(Dates.Month(1),Dates.Year(1900))

1900-01-01

In [224]:
df[:, :last_review] = coalesce.(df[:, :last_review], dummy_date);

In [225]:
df[:, :reviews_per_month] =  coalesce.(df[:, :reviews_per_month], 0);

Because the missing values are `host_name`and `name`they will be elimnated as they representa a very small percentage of the total observations

In [226]:
index_host_name = .!ismissing.(df[:, :host_name]); # 1 to rows with non missing values

In [227]:
size(df)

(48895, 16)

In [228]:
df_clean_host_name = df[index_host_name, :];

In [229]:
size(df_clean_host_name)

(48874, 16)

Now it will be checked if there are still rows with missing `name`

In [230]:
index_name = .!ismissing.(df_clean_host_name[:, :name]); # 1 to rows with non missing values

In [231]:
sum(index_name)

48858

As this number is less than the number of rows of `df_clean_host_name` cleaning is still needed

In [232]:
df_clean = df_clean_host_name[index_name, :];

In [233]:
size(df_clean)

(48858, 16)

Final check to see all variables are ok

In [234]:
 any_missing = colwise(x -> any(ismissing.(x)), df_clean);
sum(any_missing)

0

No values are missing from any of the columns. Clean dataframe is stored in new variable to prevent overwriting

In [235]:
july_2019_dataframe = df_clean;

#### Save all clean dataframes as csv

In [268]:
CSV.write("2020_october_listings_clean.csv", october_2020_dataframe)

"2020_october_listings_clean.csv"

In [269]:
CSV.write("2020_july_listings_clean.csv", july_2020_dataframe)

"2020_july_listings_clean.csv"

In [270]:
CSV.write("2019_october_listings_clean.csv", october_2019_dataframe)

"2019_october_listings_clean.csv"

In [271]:
CSV.write("2019_july_listings_clean.csv", july_2019_dataframe)

"2019_july_listings_clean.csv"

##  Variable encoding

Now that the data is clean, several columns will be encoded in order to do easier analysis:
* `neighbourhood_group`
* `neighbourhood`
* `room_type`

Maybe `last_review` could be considered to be encoded [Recently, last 3 months, never]. This is something to consider

In [243]:
dataframes = [october_2020_dataframe, july_2020_dataframe, october_2019_dataframe, july_2019_dataframe];

In [251]:
dataframe_names = ["october_2020_dataframe", "july_2020_dataframe", "october_2019_dataframe", "july_2019_dataframe"];

In [248]:
variables_to_encode = ["neighbourhood_group", "neighbourhood", "room_type" ];

In [265]:
for i in 1:length(dataframe_names)
    print("\n\n ", dataframe_names[i])
    print("\n =======================")
    
    for variable in variables_to_encode
        print("\n ", variable, "\n")
        unique_values = unique(dataframes[i][:,variable] )
        print(length(unique_values))
    end
end




 october_2020_dataframe
 neighbourhood_group
5
 neighbourhood
221
 room_type
4

 july_2020_dataframe
 neighbourhood_group
5
 neighbourhood
222
 room_type
4

 october_2019_dataframe
 neighbourhood_group
5
 neighbourhood
223
 room_type
4

 july_2019_dataframe
 neighbourhood_group
5
 neighbourhood
221
 room_type
3

* `neighbourhood_group`: Can encode directly in all of them
* `neighbourhood`: Need to see the 2 (at least) different values
* `room_type`: july 2019 only has 3 types


Next steps are to encode the three studied variables in all four datasets using the same coding pattern so the results are better understood in all. This will be accomplished with the following steps:
1. Create new column in all datasets indicating what period of time the row corresponds to (oct 2020, jul 2020...)
2. Join all datasets in the same one
3. Encode the necessary variables globally to ensure encoding is consistent across all datasets
4. Separate the data to remake initial datasets

#### 1. Create new column

#### 2. Join all datasets

#### 3. Encode all variables

In [236]:
function manyhot(column, cats=unique(col))
    result = zeros( size(column,1), size(cats,1) )
    for i in 1:length(column)
        set_of_descriptions = column[i]
        for description in set_of_descriptions
            for j in 1:length(cats) 
                if description == cats[j]
                    result[i,j] = 1
                end
            end
        end
    end
    return result
end

manyhot (generic function with 2 methods)

**`neigbourhood_group`**

**`neigbourhood`**

**`room_type`**

#### 4. Separate data by initial variable to obtain original datasets